In [69]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import Sequential, Linear, ReLU, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data.dataset import random_split

import GCL.augmentors as A
import GCL.losses as L
from GCL.models import DualBranchContrast

from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import dropout_edge, add_random_edge, to_dense_adj

from wgin_conv import WGINConv

from tqdm import tqdm
import itertools
import warnings
import sys
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os.path as osp

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
dataset_name = 'PROTEINS'
device = torch.device('cuda')
path = osp.join(osp.expanduser('~'), 'datasets')
dataset = TUDataset(path, name=dataset_name)
one_graph = dataset[0]

adj, features, labels = (to_dense_adj(one_graph.edge_index).squeeze().to(device), one_graph.x.to(device), one_graph.y.to(device))
print(adj.device)
print(features.device)
print(labels.device)

cuda:0
cuda:0
cuda:0


In [72]:
# Prepare the victim model
from gin import *

dataset_name = 'PROTEINS'
train_multiple_classifiers = False

# Hyperparams
lr = 0.01
num_layers = 3
epochs = 20
print(f'======The hyperparams: lr={lr}, num_layers={num_layers}, epochs={epochs}. On dataset:{dataset_name}======')

device = torch.device('cuda')
path = osp.join(osp.expanduser('~'), 'datasets')
dataset = TUDataset(path, name=dataset_name)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
num_features = max(dataset.num_features, 1)
num_classes = dataset.num_classes
if dataset.num_features==0 :
    print("No node feature, paddings of 1 will be used in GIN when forwarding.")


best_hyperparams = {'lr': 0.01, 'num_layer': 5, 'hidden_dim': 32, 'dropout': 0.5, 'batch_size': 128}
lr, num_layer, hidden_dim, dropout, batch_size = best_hyperparams.values()

train_val_set, eval_set = random_split(dataset, [0.9, 0.1])
dataloader_train = DataLoader(train_val_set, batch_size=batch_size, shuffle=True) # Use all train+val to train the final model
dataloader_eval = DataLoader(eval_set, batch_size=128, shuffle=False) # Do not shuffle the evaluation set to make it reproduceable



encoder_model = GIN(num_features=num_features, dim=hidden_dim, num_gc_layers=num_layer, dropout=dropout).to(device)
classifier = LogReg(hidden_dim * num_layer, num_classes).to(device)
model = GCL_classifier(encoder_model, classifier)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Train the encoder with full dataset without labels using contrastive learning
with tqdm(total=epochs, desc='(T)') as pbar:
    for epoch in range(1, epochs + 1):
        loss = train(model, dataloader_train, optimizer, scheduler)
        pbar.set_postfix({'loss': loss})
        pbar.update()

# Accuracy on the clean evaluation data
acc_clean, mask = eval_encoder(model, dataloader_eval)

# Instantiate an attacker and attack the victim model
attacker = Greedy(pn=0.05)
dataloader_eval_adv = attacker.attack(eval_set, mask)

# Accuracy on the adversarial data only
acc_adv_only, _ = eval_encoder(model, dataloader_eval_adv)

# Overall adversarial accuracy
acc_adv = acc_clean * acc_adv_only # T/all * Tadv/T = Tadv/all

print(f'(A): clean accuracy={acc_clean:.4f}, adversarial accuracy={acc_adv:.4f}')


======The hyperparams: lr=0.01, num_layers=3, epochs=20. On dataset:PROTEINS======


(T): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  8.07it/s, loss=4.96]


(A): clean accuracy=0.7568, adversarial accuracy=0.7027


In [110]:
import torch_geometric
from attacker.PGD import PGDAttack

In [111]:
attacker = PGDAttack(surrogate=model, device=device)

In [112]:
one_graph.to(device)

Data(edge_index=[2, 162], x=[42, 3], y=[1])

In [113]:
model(one_graph.x, one_graph.edge_index, one_graph.batch)

tensor([[-1.4243, -4.9847]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [114]:
fc_edge_index, edge_weight = attacker.preprocess(one_graph.x, one_graph.edge_index)

In [115]:
fc_edge_index

tensor([[ 0,  0,  0,  ..., 41, 41, 41],
        [ 1,  2,  3,  ..., 38, 39, 40]], device='cuda:0')

In [116]:
edge_weight

tensor([0., 0., 0.,  ..., 0., 0., 1.], device='cuda:0')

In [117]:
model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=edge_weight)

tensor([[-1.4243, -4.9847]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [127]:
logit = model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=edge_weight)
loss = F.cross_entropy(logit, one_graph.y)
print(loss.item())

0.028030570596456528


### 上面已经成功地进行了改造，接下来求取对于edge_weight的梯度

In [120]:
modified_weight = attacker.get_modified_weight()
logit = model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=modified_weight)
loss = F.cross_entropy(logit, one_graph.y)

loss.backward() # Calculate the gradient

In [121]:
t = 1
lr_weight = 200 / np.sqrt(t+1)
attacker.adj_changes.data.add_(lr_weight * attacker.adj_changes.grad) # Update the edge weight

attacker.adj_changes.grad.zero_() # Clear the gradient

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [125]:
n_perturbations = int(one_graph.x.shape[0] * 0.05)
fc_edge_index, modified_weight, adv_edge_index = attacker.attack(one_graph.x, one_graph.edge_index, one_graph.batch, one_graph.y, n_perturbations)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 72.49it/s]


In [126]:
adv_edge_index.shape

torch.Size([2, 166])

In [66]:
logit = model(one_graph.x, adv_edge_index, one_graph.batch, edge_weight=edge_weight)
loss = F.cross_entropy(logit, one_graph.y)
print(loss.item())

tensor([ 0.3585, -1.7292,  0.0392,  0.3737,  1.4137,  0.4494,  0.1799, -1.1697,
        -0.5752,  0.0209], device='cuda:0', grad_fn=<SliceBackward0>)

tensor(487.7718, device='cuda:0', grad_fn=<SumBackward0>)
tensor([ 0.1794,  2.4096,  2.4096,  ...,  0.4122, -3.4796, -2.2564],
       device='cuda:0', requires_grad=True)


In [38]:
def bisection(a, b, n_perturbations, epsilon):
    def func(x):
        return torch.clamp(self.adj_changes-x, 0, 1).sum() - n_perturbations

    miu = a
    while ((b-a) >= epsilon):
        miu = (a+b)/2
        # Check if middle point is root
        if (func(miu) == 0.0):
            break
        # Decide the side to repeat the steps
        if (func(miu)*func(a) < 0):
            b = miu
        else:
            a = miu
    # print("The value of root is : ","%.4f" % miu)
    return miu

# Do the projection
def projection(edge_weight, n_perturbations):
    # projected = torch.clamp(self.adj_changes, 0, 1)
    if torch.clamp(edge_weight, 0, 1).sum() > n_perturbations:
        left = (edge_weight - 1).min()
        right = edge_weight.max()
        miu = self.bisection(left, right, n_perturbations, epsilon=1e-5)
        edge_weight.data.copy_(torch.clamp(edge_weight.data - miu, min=0, max=1))
    else:
        edge_weight.data.copy_(torch.clamp(edge_weight.data, min=0, max=1))
        


In [39]:
n_perturbations = int(one_graph.x.shape[0] * 0.05)
projection(edge_weight, n_perturbations)

NameError: name 'self' is not defined

出现了一个非常奇怪的现象。一旦加入了edge_weight之后，就会出现‘RuntimeError: mat1 and mat2 must have the same dtype’。但是如果把out的形状打出来，就会发现明明没有变过，和把edge_weight=None是一样的。不能理解这个问题出现的原因。

In [31]:
# Do the attack
from attacker.PGD import PGDAttack

attacker = PGDAttack(model=encoder_classifier, nnodes=adj.shape[0], loss_type='CE', device=device).to(device)

In [28]:
adj

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 1., 0.]], device='cuda:0')

In [62]:
a = np.linspace(1,9,9).reshape((3,3))
a[~np.eye(3,dtype=bool)]

array([2., 3., 4., 6., 7., 8.])

In [64]:
matrix = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])
matrix[~torch.eye(3,dtype=bool)]

tensor([2, 3, 4, 6, 7, 8])

In [68]:
nnodes=3
torch.triu_indices(row=nnodes, col=nnodes, offset=1)

tensor([[0, 0, 1],
        [1, 2, 2]])